In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
from google.colab import files
import re

In [2]:
# Create DataFrame from csv data, adjust data types and importation parameters
df_delayed_deliveries = pd.read_csv('delayed_deliveries.csv', delimiter=';',decimal=',',
parse_dates=['Delivery Forecast'], dtype={'Delivery Number': object,'Days_of_delay': int})

In [3]:
# Creates a chart to analyze the number of delayed deliveries by State

base_model = alt.Chart(df_delayed_deliveries, title="Number of Delayed Deliveries by State").encode(
    theta=alt.Theta("count():Q", stack=True), color=alt.Color("State Destination:N", title ='State'), tooltip='State Destination'
)

pie_chart = base_model.mark_arc(outerRadius=120)

# Configure chart to display total number of delayed deliveries
text = base_model.mark_text(radius=130, size=20).encode(text=("count()"))

# Combine Pie Chart with Text
pie_chart + text

alt.LayerChart(...)

In [4]:
# Creates a interactive chart to analyze individual delayed deliveries Agregated by State

# Defines a selector for state
selector = alt.selection_single(empty='all', fields=['State Destination'])

base = alt.Chart(df_delayed_deliveries ,title="Number of Delayed Deliveries by State").properties(
    width=350,
    height=370
).add_selection(selector)


pie_chart = base.encode(
    # Count the number of delayed deliveries in state
    theta=alt.Theta("count():Q", stack=True), tooltip='State Destination',
    color=alt.condition(selector,'State Destination:N',alt.value('lightgray'), legend=alt.Legend(orient="top"))
).mark_arc(outerRadius=110).properties(
    title='Overview of Delayed Deliveries by State'
)


points_chart = base.mark_circle(size=100).encode(
    x = alt.X('State Destination',title='State/UF',axis=alt.Axis(labelAngle=0),
              sort=alt.EncodingSortField(field="State Destination", op="count", order='descending')),
    y = alt.Y('Days of Delay:Q',title='Days of Delay', scale=alt.Scale(domain=(0,30),clamp=True)),
    color=alt.condition(selector,'State Destination:N',alt.value('lightgray')),
    tooltip=('Delivery Number','Payer','Days of Delay')
).interactive()


base_pie = alt.Chart(df_delayed_deliveries, title="Overview delayed deliveries by State").encode(
    theta=alt.Theta("count():Q", stack=True),
    color=alt.condition(selector,'State Destination:N',
    alt.value('lightgray')), tooltip='State Destination'
)

# Add number of delayed deliveries to the pizza chart
pie_text_values = base_pie.mark_text(radius=120, size=15).encode(text=("count()"))

# Combine charts
points_chart | pie_chart + pie_text_values

alt.HConcatChart(...)

In [5]:
# This chart agregates the states with more delayed deliveries at the bottom of the heatmap
# Highlighting the bottlenecked of deliveries

color_scale = alt.Scale(domain=[1, 10], range=['#b3ffb3', '#ff3300'])

heatmap = alt.Chart(df_delayed_deliveries, title='Heatmap of Delayed Deliveries').mark_rect().encode(
    alt.X('Days of Delay:Q', title='Days of Delay', bin=alt.Bin(step=3),scale=alt.Scale(domain=(0,55),clamp=True)),
    alt.Y('State Destination', title='State',sort=alt.EncodingSortField(field="State Destination", op="count", order='ascending')),
    alt.Color('count():Q', scale=alt.Scale(domainMid = 5,range=['#b3ffb3', '#ff3300']), title='Nº Of Deliveries'),
    tooltip=[alt.Tooltip('count()', title='Deliveries Delayed')]
).properties(width=600,height=300)

# Add text of number of delayed days to heatmap
text = heatmap.mark_text(baseline='middle').encode(
    text='count()',
    color=alt.value('white')
)
heatmap + text

alt.LayerChart(...)

In [6]:
# This chart displays 2 important financial indicators of clients

# The bar chart is the first financial indicator, it shows how much the ShippmentCost is worth vs the Cargo Value in %
# The red line is the second indicator, it represent how much $ is charged for every 1 Kilo shipped

client_analysis_weight_and_value = pd.read_csv('clients_indicators.csv', delimiter=';',decimal=',')
client_analysis_weight_and_value = client_analysis_weight_and_value.sort_values(by=['Shipping_Value_on_Product_Value'], ascending=False)
cutPointValue = client_analysis_weight_and_value['Shipping_Value_on_Product_Value'].quantile([0.95]).values[0] # remove outliers

client_analysis_weight_and_value = client_analysis_weight_and_value
[client_analysis_weight_and_value['Shipping_Value_on_Product_Value'] < cutPointValue]

# Select only the top 20 biggest customers
client_analysis_weight_and_value = client_analysis_weight_and_value.head(20)

base = alt.Chart(client_analysis_weight_and_value).encode(x='Client:N')

bar = base.mark_bar().encode(
    alt.Y('Shipping_Value_on_Product_Value:Q', title='% of Shipping Value over Product Value',axis=alt.Axis(labelExpr="datum.value + '%'"))
)

line =  base.mark_line(color='red').encode(
    y='Shipping_Value/KG:Q',
    tooltip=[alt.Tooltip('Shipping_Value/KG', title='Shipping Value by KG')]
)

# Combine Charts
(bar + line).properties(width=600)

alt.LayerChart(...)

In [7]:
# This chart allows for vizualization of Key indicator of each of the Company Branches
# Including Number of Deliveris, Shipped Value, number of Volumes and Average Shipping of Deliveries

df_selector_graph = pd.read_csv('clients_indicators.csv', delimiter=';',decimal=',')

# Filters the data on the chart, grouping by Subsidiary_Responsible and summing the results of interest variables
df_selector_graph = df_selector_graph.groupby(['Subsidiary_Responsible'])[['Number_of_Deliveries',
'Total_Shipped_Value','Number_of_volumes','Average_Shipping_Value','Total_KG']].sum().reset_index()
df_selector_graph = pd.melt(df_selector_graph, id_vars=['Subsidiary_Responsible'],
value_vars=[ 'Number_of_Deliveries','Total_Shipped_Value', 'Number_of_volumes','Average_Shipping_Value','Total_KG'])

# Creates the Dropdown to select Variable
dropdown = alt.binding_select(options=list(df_selector_graph.variable.drop_duplicates()))
selection = alt.selection_single(fields=['variable'], bind=dropdown, name='Select')

alt.Chart(df_selector_graph).mark_rect().encode(
    alt.X('value:Q', title='Value of variable'),
    alt.Y('Subsidiary_Responsible:N', title='Subsidiary'),
    alt.Color('value:Q', scale=alt.Scale(domainMid= 1000 ,range=['#ffcc00', '#66ff66']), title='Nº Of Deliveries'),
    tooltip=[alt.Tooltip('value:Q', title='Value ')]
).add_selection(
    selection
).transform_filter(
    selection
).properties(title="Performance Indicators of Branchs")

alt.Chart(...)

In [9]:
# This chart is used for financial analysys of clients
# Tt compares shipping value of clients with others financial indicators

# This Graph also serves as an example of how to display multiple variables

df_all_clients = pd.read_csv('clients_indicators.csv', delimiter=';',decimal=',')

df_all_clients = df_all_clients.sort_values(by=['Total_KG'], ascending=False) # Order by biggest clients
df_all_clients['Shipping_Value_on_Product_Value_Percentual'] = df_all_clients['Shipping_Value_on_Product_Value']/100 # used to display percentage
# Selects onli top 30 clients
df_top_30_clients = df_all_clients.head(30)

max_domain_value = df_top_30_clients['Shipping_Value_on_Product_Value'].quantile([0.95]).values[0] # remove outliers
max_domain_kg = df_top_30_clients['Shipping_Value/KG'].quantile([0.95]).values[0] # remove outliers
client_performance = (df_top_30_clients["Shipping_Value/KG"] + df_top_30_clients["Shipping_Value_on_Product_Value"]).values.tolist()

# Brush for selection
brush = alt.selection(type='interval')

# Creates Scatter Plot
financial_report_chart = alt.Chart(df_top_30_clients).mark_point(filled= True, shape = 'circle').encode(
    alt.X('Shipping_Value/KG:Q',title='Freight Cost ($) per 1 KG', scale=alt.Scale(zero=False, clamp=True, domainMax = max_domain_kg)),
    alt.Y('Shipping_Value_on_Product_Value:Q', axis=alt.Axis(labelExpr="datum.value + '%'") ,
          title='Freight Cost VS Merchandise Value in %',scale=alt.Scale(zero=False, clamp=True)),
    alt.Size('Total_KG:Q',scale=alt.Scale(range=[50, 700]), legend =None), #,legend=alt.Legend(format='.2f')

    color=alt.condition(brush, 'Suggested_adjustment:Q', alt.value('grey'),scale=alt.Scale(domainMin =5, # adjust color to be lighter
                        range=['#ff3300', '#1aff1a']), title='Adjustment Value Requeried (%)',sort='descending'),

    tooltip=[ 'Client','Subsidiary_Responsible',alt.Tooltip('Shipping_Value/KG',title='Freight Cost ($) per 1 KG'),
             alt.Tooltip('Shipping_Value_on_Product_Value_Percentual',title='Shipping Cost VS Product Value', format=".2%"),
              alt.Tooltip('Total_Shipped_Value',title='Total Cargo Shipped $', format=",.2f"), alt.Tooltip('Total_KG', format=",.2f")]
).add_selection(brush).properties(width=700,height=500).properties(title="Financial Analysis Report - Top 30 Clients")


# Base chart for data tables
ranked_text = alt.Chart(df_top_30_clients).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<10
)

# Define Data Table Itens
client_name = ranked_text.encode(text='Client:N').properties(title='Client')
subsidiary  = ranked_text.encode(text='Subsidiary_Responsible:N').properties(title='Subsidiary')
number_of_deliveries = ranked_text.encode(text='Number_of_Deliveries:N').properties(title='Number of Deliveries')
total_shipped_weight = ranked_text.encode(text=alt.Text("Total_KG:N", format=",.2f")).properties(title='Total KG')
average_shipped_value = ranked_text.encode(text='Average_Shipping_Value:N').properties(title='Average Freight ($)')
number_of_volumes = ranked_text.encode(text=alt.Text("Number_of_volumes:N", format="")).properties(title='Total Packages Delivered')
default_charged_value = ranked_text.encode(text=alt.Text("Suggested_adjustment:N", format="")).properties(title='Recommended Value Adjustment (%)')

financial_report_table = alt.hconcat(client_name, subsidiary , number_of_deliveries, total_shipped_weight,
                   average_shipped_value, number_of_volumes, default_charged_value) # Combine data tables

# Concat all the Charts
alt.vconcat(
    financial_report_chart,
    financial_report_table
).resolve_legend(
    color="independent"
).configure_view(strokeWidth=0)

alt.VConcatChart(...)